# Lab 05. Deep Learning - MLP, CNN, RNN

## Table of Contents
- Perceptron & MLP
- CNN
- RNN

In [ ]:
import os
import copy
from os.path import join

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

np.random.seed(0)

## 1. Perceptron

In [ ]:
X = np.array([[2, 2], [1, 3], [2, 3], [5, 3], [2, 4], [3, 4],
              [6, 4], [1, 5], [5, 5], [4, 6], [6, 6], [5, 7]])
Y = np.array([0, 0, 0, 1, 0, 0,
              1, 0, 1, 1, 1, 1])

# 위 X,Y 데이터를 점 그래프로 표현 / 라이브러리 이름 matplotlib
plt.scatter(X[:, 0], X[:, 1], c=Y)

In [ ]:
from sklearn.linear_model import Perceptron

# 모델 생성
model = Perceptron(tol=1e-3, max_iter=100)

# 모델 학습
model.fit(X, Y)

In [ ]:
# 2차원 공간의 점들의 좌표. x와 y의 최대, 최소값을 구하여, 그래프의 가로축 세로축 길이를 결정.
h = 0.02

# horizontal은 가로축, vertical는 세로축을 의미. (그래프를 그릴 평면의 사이즈 설정)
# X[:, 0]은 좌표 값들 중 가로축의 값.
# X[:, 1]은 좌표 값들 중 세로축의 값.
horizontal_min, horizontal_max = X[:, 0].min() - 1, X[:, 0].max() + 1
vertical_min, vertical_max = X[:, 1].min() - 1, X[:, 1].max() + 1

# 0.02 간격으로 min값과 max 사이 matrix를 생성하기 위한 작업.
xx, yy = np.meshgrid(np.arange(horizontal_min, horizontal_max, h),
                     np.arange(vertical_min, vertical_max, h))

print(xx)
print(xx.shape)

print()
print(yy)
print(yy.shape)

In [ ]:
# 공간 상의 점들(영역)에 대한 모델의 예측 값들을 구함.
# 평면에 색을 칠하기 위해 0.02 간격의 점들에 대해 모두 예측값을 구한 것.(하늘색과 주황색 평면)
fig, ax = plt.subplots()

# ravel() : 1차원으로 푸는것. == 'reshape(-1, 1).squeeze()'
print(xx.ravel().shape)
print()

# c_[array1, array2] : column_stack  <-> row stack (r_[array1, array2])
'''
[1,    6]     [1, 6]
[2, +  7]  =  [2, 7]
[3,    8]     [3, 8]

'''

xxx = np.c_[xx.ravel(), yy.ravel()]
print(xxx)
print()
print(xxx.shape)

Z = model.predict(xxx)

# 예측 값(0 or 1)에 색을 넣고 그래프에 (영역을) 그린다.
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z, cmap=plt.cm.Paired)
ax.axis('off') # axis on/off

# 본래 우리가 예측하려던 입력 데이터(점)를 그린다.
ax.scatter(X[:, 0], X[:, 1], c=Y)
ax.set_title('Perceptron')


## 2. MLP with XOR Problem

---

Perceptron은 단순한 데이터, 즉 선형 분리가 가능한 (Linearly separable)한 데이터에서는 아주 잘 동작합니다.<br>
하지만 데이터가 선형으로 분리가 가능하지 않다면 동작하지 않습니다.

대표적인 문제가 XOR Problem 입니다.<br>
XOR는 Exclusive OR의 약자로 두 입력 중 오직 하나만이 참일 때 참, 그 외에는 거짓이 되는 연산입니다.

<h2 align="center">
  <img src='https://miro.medium.com/max/300/0*LYlt6CZJHOJkNRHJ.' height=147 width=300/>
</h2>

<!-- ![XOR](https://miro.medium.com/max/300/0*LYlt6CZJHOJkNRHJ.) -->



아래 두 블록의 코드를 실행해봅니다.<br>
어떤 직선을 그어도 입력에 대해 XOR를 참과 거짓으로 분류할 수 없습니다.<br>

하나의 직선 혹은 하나의 Perceptron으로 분류할 수 없다면 여러 개를 사용해서 분류할 수 있을까요?<br>
Multilayer Perceptron (MLP)로 XOR 문제를 풀어보도록 하겠습니다.



In [ ]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0.0], [1.0], [1.0], [0.0]])

In [ ]:
# True 인 경우 빨간 o로, False인 경우 파란 x로 그래프 표현
XOR_TRUE = x[y[:, 0]==1, :]
XOR_FALSE = x[y[:, 0]==0, :]

# print(XOR_TRUE)
# print(XOR_FALSE)

plt.scatter(XOR_TRUE[:, 0], XOR_TRUE[:, 1], color='r', marker='o', s=300)
plt.scatter(XOR_FALSE[:, 0], XOR_FALSE[:, 1], color='b', marker='x', s=300)

### XOR 문제 해결


구현할 모델의 개요는 아래와 같은 MLP 입니다.

<h2 align="left">
  <img src='https://miro.medium.com/max/630/1*qA_APGgbbh0QfRNsRyMaJg.png' height=250 width=400/>
</h2>

[이미지 출처: Neural Network XOR Application and Fundamentals](https://becominghuman.ai/neural-network-xor-application-and-fundamentals-6b1d539941ed)



In [ ]:
# Scikit-learn(sklearn) 버전
from sklearn.neural_network import MLPClassifier

# 1 hidden layer with 2 hidden nodes
hidden_layers = (2, )

# 과연 어느 정도의 learning rate로 몇 iteration을 학습해야
# XOR을 풀 수 있는 지 확인해보세요!
solver = 'sgd'
learning_rate = 1.0
max_iter = 10
verbose = True # 매 epoch 마다 학습 양상(loss 변화) 출력 여부

# 모델 생성
# activation='logistic' == sigmoid function
model = MLPClassifier(hidden_layer_sizes=hidden_layers, solver=solver, activation='logistic',
                      learning_rate_init =learning_rate, max_iter=max_iter, verbose=verbose,
                      n_iter_no_change=1000, random_state=1)

In [ ]:
# 모델 학습
model.fit(x, y)

In [ ]:
# 학습 결과 확인
pred = model.predict(x)
for i in range(len(x)):
    print('Pred: %d, Answer: %d' % (pred[i], y[i]))

In [ ]:
W1, W2 = model.coefs_
b1, b2 = model.intercepts_

# print(W1.shape)
# print(W2.shape)
# print(b1.shape)
# print(b2.shape)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

hidden_layer = sigmoid(np.matmul(x, W1) + b1)
output_layer = sigmoid(np.matmul(hidden_layer, W2) + b2)

h1 = hidden_layer[:, 0]
h2 = hidden_layer[:, 1]
print('[Hidden Node Values]')
print('Input \t H1   H2  Pred Answer')
print(x[0], ' ', '%.2f' % h1[0], '%.2f' % h2[0], '  %d' % pred[0], '   %d' % y[0])
print(x[1], ' ', '%.2f' % h1[1], '%.2f' % h2[1], '  %d' % pred[1], '   %d' % y[1])
print(x[2], ' ', '%.2f' % h1[2], '%.2f' % h2[2], '  %d' % pred[2], '   %d' % y[2])
print(x[3], ' ', '%.2f' % h1[3], '%.2f' % h2[3], '  %d' % pred[3], '   %d' % y[3])

# print(W2)
# print(b2)
# print(np.matmul(hidden_layer, W2))
# print(output_layer)

## 3. Convolutional Neural Network
이미지를 분류하는 모델을 만들어 보겠습니다.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
torch.backends.cudnn.deterministic=True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # cuda
print(device)

### 데이터셋 불러오기

In [ ]:
# Dataset 다운로드
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# print(type(trainset))

### 이미지 몇 개만 확인해보기

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# image를 출력하는 함수
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# trainset image를 갖고 와서 image 4개를 확인해본다.
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

### CNN 정의하기

In [ ]:
# CNN 모델 정의
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)

### 손실함수와 optimizer 정의하기

In [ ]:
# 손실함수와 optimizer 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### CNN 학습하기

In [ ]:
# CNN 학습
for epoch in range(2):  # 시간 관계상 2 epochs만 학습한다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # data => [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # parameter의 gradient 초기화
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

### 모델 저장하기

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

### Test 이미지 몇 개만 확인해보기

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

### 위에 출력한 Test 이미지에 대한 예측

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

### 전체 test 데이터에 대해 accuracy 계산

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

### Class 별 모델 평가

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

### Fashion-MNIST로 CNN을 모델링하기
**None 부분을 채워주세요! <br>**
> 첫 번째 / 두 번째 conv. layer 출력 채널 수: 5개, 10개 <br>
Conv. layer의 kernel 크기: 5 X 5 <br>
Hint: Fashion-MNIST는 흑백 이미지라서 채널 수가 1개 <br>
사용할 함수: nn.Conv2d(), nn.Linear(), view()


In [ ]:
class FashionNet(nn.Module):
    def __init__(self):
        self.conv1 = None
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = None
        self.fc1 = None
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = None

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = None
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
#@title 정답코드

class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 5)
        self.fc1 = nn.Linear(10 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 10 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# 데이터셋 불러오기
trainset = torchvision.datasets.FashionMNIST("./data", download=True,
                                              transform=transforms.Compose([transforms.ToTensor()]))
testset = torchvision.datasets.FashionMNIST("./data", download=True, train=False,
                                             transform=transforms.Compose([transforms.ToTensor()]))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('T-shirt/Top', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

print(next(iter(trainloader))[0].shape)

In [ ]:
# 이미지 몇 개만 확인해보기
import matplotlib.pyplot as plt
import numpy as np

# image를 출력하는 함수
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# trainset image를 갖고 와서 image 4개를 확인해본다.
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# 모델 정의하기
net = FashionNet().to(device)

In [ ]:
# 손실함수와 optimizer 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# 모델 학습하기
# CNN 학습
for epoch in range(2):  # 시간 관계상 2 epochs만 학습한다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # data => [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # parameter의 gradient 초기화
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# 테스트 이미지 몇 개만 확인해보기
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# 모델 저장하기
PATH = './fasion_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
# 모델 불러오기
net = FashionNet()
net.load_state_dict(torch.load(PATH))

In [ ]:
# 위의 이미지에 대한 예측
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
# 전체 테스트 데이터에 대해 정확도 계산
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# 클래스 별 모델 평가
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))